In [131]:
%reset -f
import pandas as pd
import numpy as np
import matplotlib as mp
import seaborn as sns
import csv
import re
import gc
import sys, os, random

import matplotlib.pyplot as plt # for plotting

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

os.environ['PYTHONHASHSEED'] = '0'
np.random.seed(37)
random.seed(17)

pd.set_option('display.max_rows',1000)

root = '/Users/schwalmdaniel/github/kaggle/home-credit-default-risk'
#root = 'e:/kaggle/home-credit-default-risk'

train=pd.read_csv(root + "/application_train.csv")
test=pd.read_csv(root + "/application_test.csv")
bureau=pd.read_csv(root + "/bureau.csv")
'''bureau_balance=pd.read_csv(root + "/bureau_balance.csv")
POS_CASH_balance=pd.read_csv(root + "/POS_CASH_balance.csv")
credit_card_balance=pd.read_csv(root + "/credit_card_balance.csv")
previous_application=pd.read_csv(root + "/previous_application.csv")
installments_payments=pd.read_csv(root + "/installments_payments.csv")'''

# have a look at the ds
train.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [132]:
bureau.head()

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY
0,215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,NaN,0,91323.0,0.0,NaN,0.0,Consumer credit,-131,NaN
1,215354,5714463,Active,currency 1,-208,0,1075.0,NaN,NaN,0,225000.0,171342.0,NaN,0.0,Credit card,-20,NaN
2,215354,5714464,Active,currency 1,-203,0,528.0,NaN,NaN,0,464323.5,NaN,NaN,0.0,Consumer credit,-16,NaN
3,215354,5714465,Active,currency 1,-203,0,NaN,NaN,NaN,0,90000.0,NaN,NaN,0.0,Credit card,-16,NaN
4,215354,5714466,Active,currency 1,-629,0,1197.0,NaN,77674.5,0,2700000.0,NaN,NaN,0.0,Consumer credit,-21,NaN


In [133]:
bureau_credit = pd.DataFrame(bureau[['SK_ID_CURR', 'CREDIT_ACTIVE']])
bureau_overdue = pd.DataFrame(bureau[['SK_ID_CURR', 'CREDIT_DAY_OVERDUE']])
bureau_currency = pd.DataFrame(bureau[['SK_ID_CURR', 'CREDIT_CURRENCY']])
bureau_prolong = pd.DataFrame(bureau[['SK_ID_CURR', 'CNT_CREDIT_PROLONG']])

bureau_credit_active = bureau_credit[bureau_credit['CREDIT_ACTIVE'] == 'Active'].\
    groupby('SK_ID_CURR').CREDIT_ACTIVE.agg(['count']).reset_index()
bureau_credit_closed = bureau_credit[bureau_credit['CREDIT_ACTIVE'] == 'Closed'].\
    groupby('SK_ID_CURR').CREDIT_ACTIVE.agg(['count']).reset_index()
bureau_credit_overdue = bureau_overdue.groupby('SK_ID_CURR').CREDIT_DAY_OVERDUE.agg([
        'min', 
        'max', 
        'mean', 
        'std']).reset_index()
bureau_credit_overdue.fillna(0,inplace=True)
bureau_credit_prolong = bureau_prolong.groupby('SK_ID_CURR').CNT_CREDIT_PROLONG.agg([
        'min', 
        'max', 
        'mean', 
        'std']).reset_index()
bureau_currency_count = bureau_currency.groupby('SK_ID_CURR').CREDIT_CURRENCY.agg(['count']).reset_index()

train=train.merge(bureau_credit_active,on='SK_ID_CURR', how='left')
test=test.merge(bureau_credit_active,on='SK_ID_CURR', how='left')
train.rename(columns={'count': 'bureau_credit_active_count'}, inplace=True)
test.rename(columns={'count': 'bureau_credit_active_count'}, inplace=True)

train=train.merge(bureau_currency_count,on='SK_ID_CURR', how='left')
test=test.merge(bureau_currency_count,on='SK_ID_CURR', how='left')
train.rename(columns={'count': 'bureau_currency_count'}, inplace=True)
test.rename(columns={'count': 'bureau_currency_count'}, inplace=True)

train=train.merge(bureau_credit_closed,on='SK_ID_CURR', how='left')
test=test.merge(bureau_credit_closed,on='SK_ID_CURR', how='left')
train.rename(columns={'count': 'bureau_credit_closed_count'}, inplace=True)
test.rename(columns={'count': 'bureau_credit_closed_count'}, inplace=True)

train=train.merge(bureau_credit_overdue,on='SK_ID_CURR', how='left')
test=test.merge(bureau_credit_overdue,on='SK_ID_CURR', how='left')
train.rename(columns={'min': 'bureau_credit_overdue_min','max': 'bureau_credit_overdue_max',
                      'mean': 'bureau_credit_overdue_mean','std': 'bureau_credit_overdue_std'}, inplace=True)
test.rename(columns={'min': 'bureau_credit_overdue_min','max': 'bureau_credit_overdue_max',
                      'mean': 'bureau_credit_overdue_mean','std': 'bureau_credit_overdue_std'}, inplace=True)

train=train.merge(bureau_credit_prolong,on='SK_ID_CURR', how='left')
test=test.merge(bureau_credit_prolong,on='SK_ID_CURR', how='left')
train.rename(columns={'min': 'bureau_credit_prolong_min','max': 'bureau_credit_prolong_max',
                      'mean': 'bureau_credit_prolong_mean','std': 'bureau_credit_prolong_std'}, inplace=True)
test.rename(columns={'min': 'bureau_credit_prolong_min','max': 'bureau_credit_prolong_max',
                      'mean': 'bureau_credit_prolong_mean','std': 'bureau_credit_prolong_std'}, inplace=True)
train['bureau_credit_overdue_min'].fillna(0,inplace=True)
train['bureau_credit_overdue_max'].fillna(0,inplace=True)
train['bureau_credit_overdue_mean'].fillna(0,inplace=True)
train['bureau_credit_overdue_std'].fillna(0,inplace=True)
train['bureau_credit_prolong_min'].fillna(0,inplace=True)
train['bureau_credit_prolong_max'].fillna(0,inplace=True)
train['bureau_credit_prolong_mean'].fillna(0,inplace=True)
train['bureau_credit_prolong_std'].fillna(0,inplace=True)
train['bureau_credit_active_count'].fillna(0,inplace=True)
train['bureau_credit_closed_count'].fillna(0,inplace=True)
train['bureau_currency_count'].fillna(0,inplace=True)
test['bureau_credit_overdue_min'].fillna(0,inplace=True)
test['bureau_credit_overdue_max'].fillna(0,inplace=True)
test['bureau_credit_overdue_mean'].fillna(0,inplace=True)
test['bureau_credit_overdue_std'].fillna(0,inplace=True)
test['bureau_credit_prolong_min'].fillna(0,inplace=True)
test['bureau_credit_prolong_max'].fillna(0,inplace=True)
test['bureau_credit_prolong_mean'].fillna(0,inplace=True)
test['bureau_credit_prolong_std'].fillna(0,inplace=True)
test['bureau_credit_active_count'].fillna(0,inplace=True)
test['bureau_credit_closed_count'].fillna(0,inplace=True)
test['bureau_currency_count'].fillna(0,inplace=True)

train.head()
                            




,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,bureau_currency_count,bureau_credit_closed_count,bureau_credit_overdue_min,bureau_credit_overdue_max,bureau_credit_overdue_mean,bureau_credit_overdue_std,bureau_credit_prolong_min,bureau_credit_prolong_max,bureau_credit_prolong_mean,bureau_credit_prolong_std
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,8.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,4.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [157]:
bureau_credit_prolong['mean'].value_counts()

0.000000    297015
0.250000       772
0.200000       698
0.166667       696
0.333333       660
0.142857       626
0.125000       598
0.500000       593
0.111111       481
0.100000       420
0.090909       364
0.083333       343
1.000000       328
0.076923       248
0.071429       195
0.066667       148
0.062500       143
0.058824       118
0.400000       108
0.285714       108
0.666667       104
0.055556        98
0.222222        89
0.181818        75
0.052632        59
0.050000        56
0.153846        54
0.047619        53
2.000000        50
0.045455        34
0.133333        32
0.043478        29
0.041667        25
0.428571        25
0.750000        24
0.600000        21
0.375000        21
0.117647        20
0.040000        19
0.105263        16
0.095238        15
0.038462        15
0.035714        15
0.272727        12
0.300000        11
0.037037        11
1.500000        10
0.214286         9
0.230769         8
0.176471         6
3.000000         6
0.555556         5
0.800000    

In [134]:
test.shape

(48744, 132)

In [135]:
train['TARGET'].value_counts()

# it is an unbalanced data, 8.5% of the target is 1, so the baseline is around 92%

0    282686
1     24825
Name: TARGET, dtype: int64

In [136]:
# 'NAME_CONTRACT_TYPE', 2 values, converting to 0/1
train['NAME_CONTRACT_TYPE'] = train['NAME_CONTRACT_TYPE'].apply(lambda x: 0 if x == 'Cash loans' else 1)
test['NAME_CONTRACT_TYPE'] = test['NAME_CONTRACT_TYPE'].apply(lambda x: 0 if x == 'Cash loans' else 1)

In [137]:
# 'CODE_GENDER', drop XNA as only 4 rows, convert the rest to 0/1
train = train[train['CODE_GENDER'] != 'XNA']
train['CODE_GENDER'] = train['CODE_GENDER'].apply(lambda x: 0 if x == 'F' else 1)
test['CODE_GENDER'] = test['CODE_GENDER'].apply(lambda x: 0 if x == 'F' else 1)

In [138]:
# FLAG_OWN_CAR
train['FLAG_OWN_CAR'] = train['FLAG_OWN_CAR'].apply(lambda x: 1 if x == 'Y' else 0)
test['FLAG_OWN_CAR'] = test['FLAG_OWN_CAR'].apply(lambda x: 1 if x == 'Y' else 1)

In [139]:
# FLAG_OWN_REALTY
train['FLAG_OWN_REALTY'] = train['FLAG_OWN_REALTY'].apply(lambda x: 1 if x == 'Y' else 0)
test['FLAG_OWN_REALTY'] = test['FLAG_OWN_REALTY'].apply(lambda x: 1 if x == 'Y' else 1)

In [140]:
# where null it should be dropped or mean or average income/annuity

avgAnnuityRate = (train['AMT_ANNUITY']/train['AMT_CREDIT']).mean()
train['AMT_ANNUITY'].fillna(avgAnnuityRate * train['AMT_CREDIT'],inplace=True)
test['AMT_ANNUITY'].fillna(avgAnnuityRate * train['AMT_CREDIT'],inplace=True)


In [141]:
#  where null mean or average income / goods price
goodsPriceMean = train['AMT_GOODS_PRICE'].mean()
train['AMT_GOODS_PRICE'].fillna(goodsPriceMean,inplace=True)
test['AMT_GOODS_PRICE'].fillna(goodsPriceMean,inplace=True)

In [142]:
#  categorical, dummify, where null either unknown or most frequent
train['NAME_TYPE_SUITE'].fillna('Unaccompanied',inplace=True)
test['NAME_TYPE_SUITE'].fillna('Unaccompanied',inplace=True)

In [143]:
# numeric, the older the worse, where null check own car
train['OWN_CAR_AGE'].fillna(100,inplace=True)
test['OWN_CAR_AGE'].fillna(100,inplace=True)
train['OWN_CAR_AGE'] = train['OWN_CAR_AGE'] * -1
test['OWN_CAR_AGE'] = test['OWN_CAR_AGE'] * -1
train['OWN_CAR_AGE'].describe()

count    307507.000000
mean        -70.092882
std          42.238518
min        -100.000000
25%        -100.000000
50%        -100.000000
75%         -15.000000
max          -0.000000
Name: OWN_CAR_AGE, dtype: float64

In [144]:
# numeric, drop where it is null
train = train[train['CNT_FAM_MEMBERS'] > 0]
test = test[test['CNT_FAM_MEMBERS'] > 0]

In [145]:
# fill null with mean for _1, _2, _3
'''mean1 = train['EXT_SOURCE_1'].mean()
mean2 = train['EXT_SOURCE_2'].mean()
mean3 = train['EXT_SOURCE_3'].mean()
train['EXT_SOURCE_1'].fillna(mean1,inplace=True)
train['EXT_SOURCE_2'].fillna(mean2,inplace=True)
train['EXT_SOURCE_3'].fillna(mean3,inplace=True)
test['EXT_SOURCE_1'].fillna(mean1,inplace=True)
test['EXT_SOURCE_2'].fillna(mean2,inplace=True)
test['EXT_SOURCE_3'].fillna(mean3,inplace=True)'''
train['EXT_SOURCE_1'].fillna(0,inplace=True)
train['EXT_SOURCE_2'].fillna(0,inplace=True)
train['EXT_SOURCE_3'].fillna(0,inplace=True)
test['EXT_SOURCE_1'].fillna(0,inplace=True)
test['EXT_SOURCE_2'].fillna(0,inplace=True)
test['EXT_SOURCE_3'].fillna(0,inplace=True)


In [146]:
# fill with mean all _AVG, _MEDI, _MODE
for col in train.columns.tolist():
    if (col.endswith('_AVG') or col.endswith('_MEDI') or col.endswith('_MODE')) and col not in ['FONDKAPREMONT_MODE','HOUSETYPE_MODE',
                    'WALLSMATERIAL_MODE','EMERGENCYSTATE_MODE']: 
        #print (col)
        mean = train[col].mean()
        train[col].fillna(mean,inplace=True)
        test[col].fillna(mean,inplace=True)
        

In [147]:
# EMERGENCYSTATE_MODE
train['EMERGENCYSTATE_MODE'] = train['EMERGENCYSTATE_MODE'].apply(lambda x: 1 if x == 'Y' else 0)
test['EMERGENCYSTATE_MODE'] = test['EMERGENCYSTATE_MODE'].apply(lambda x: 1 if x == 'Y' else 1)


In [148]:
#  Fill none with mean or median for all circle
for col in train.columns.tolist():
    if col.endswith('_CIRCLE'):
        mean = train[col].mean()
        train[col].fillna(mean,inplace=True)
        test[col].fillna(mean,inplace=True)


In [149]:
# negative numeric, drop where it is null
train['DAYS_LAST_PHONE_CHANGE'].fillna(0,inplace=True)
test['DAYS_LAST_PHONE_CHANGE'].fillna(0,inplace=True)


In [150]:
# all amt_credit req, maybe bin it
train['AMT_REQ_CREDIT_BUREAU_YEAR'].mean()

for col in train.columns.tolist():
    if 'AMT_REQ_CREDIT_BUREAU_' in col:
        #print (col)
        mean = train[col].mean()
        train[col].fillna(mean,inplace=True)
        test[col].fillna(mean,inplace=True)


In [151]:
gc.collect()

0

In [152]:
train_objs_num = len(train)
dataset = pd.concat(objs=[train, test], axis=0)
dataset = pd.get_dummies(dataset, 
    columns = ['NAME_TYPE_SUITE','NAME_INCOME_TYPE','NAME_EDUCATION_TYPE','NAME_FAMILY_STATUS','NAME_HOUSING_TYPE',
            'OCCUPATION_TYPE','WEEKDAY_APPR_PROCESS_START','ORGANIZATION_TYPE','FONDKAPREMONT_MODE',
            'HOUSETYPE_MODE','WALLSMATERIAL_MODE'],prefix_sep='__')
train = dataset[:train_objs_num]
test = dataset[train_objs_num:]
train.shape

(307505, 250)

In [153]:
from sklearn.model_selection import train_test_split

X = train.drop(['SK_ID_CURR','TARGET'], axis=1)
y = train['TARGET']
X_test = test.drop(['SK_ID_CURR','TARGET'], axis=1)

#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [154]:
import lightgbm as lgb
from sklearn.metrics import mean_squared_error
from sklearn.metrics import roc_auc_score, accuracy_score
from sklearn.model_selection import KFold, RepeatedKFold

import gc
import csv

cnt = 0
p_buf = []
n_splits = 2
n_repeats = 1
kf = RepeatedKFold(
    n_splits=n_splits, 
    n_repeats=n_repeats, 
    random_state=0)
auc_buf = []   

params = {
        'boosting_type': 'gbdt',
        'objective': 'binary',
        'metric': 'auc',
        'max_depth': 12,
        'num_leaves': 31,
        'learning_rate': 0.025,
        'feature_fraction': 0.85,
        'bagging_fraction': 0.85,
        'bagging_freq': 5,
        'verbose': 0,
        'num_threads': 8,
        'lambda_l2': 1.5,
        'min_gain_to_split': 0,
    }  

for train_index, valid_index in kf.split(X):
    print('Fold {}/{}'.format(cnt + 1, n_splits))
    
    model = lgb.train(
        params,
        lgb.Dataset(X.loc[train_index], y.loc[train_index], feature_name=X.columns.tolist()),
        num_boost_round=10000,
        valid_sets=[lgb.Dataset(X.loc[valid_index], y.loc[valid_index])],
        early_stopping_rounds=100,
        verbose_eval=100,
    )

    if cnt == 0:
        importance = model.feature_importance()
        model_fnames = model.feature_name()
        tuples = sorted(zip(model_fnames, importance), key=lambda x: x[1])[::-1]
        tuples = [x for x in tuples if x[1] > 0]
        print('Important features:')
        print(tuples[:200])

    p = model.predict(X.loc[valid_index], num_iteration=model.best_iteration)
    #auc = roc_auc_score(y.loc[valid_index], p)

    #print('{} AUC: {}'.format(cnt, auc))

    p = model.predict(X_test, num_iteration=model.best_iteration)
    if len(p_buf) == 0:
        p_buf = np.array(p)
    else:
        p_buf += np.array(p)
    #auc_buf.append(auc)

    cnt += 1
    if cnt > 0: # Comment this to run several folds
        break
    
    del model
    gc.collect

#auc_mean = np.mean(auc_buf)
#auc_std = np.std(auc_buf)
#print('AUC = {:.6f} +/- {:.6f}'.format(auc_mean, auc_std))

preds = p_buf/cnt

subm = pd.DataFrame()
subm['SK_ID_CURR'] = test['SK_ID_CURR']
subm['TARGET'] = preds
subm.to_csv('home-default-risk_lgbm.csv', index=False,quoting=csv.QUOTE_NONNUMERIC)
subm.head()

Fold 1/2


/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:40: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:42: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike


Training until validation scores don't improve for 100 rounds.
[100]	valid_0's auc: 0.736084
[200]	valid_0's auc: 0.748885
[300]	valid_0's auc: 0.754809
[400]	valid_0's auc: 0.757104
[500]	valid_0's auc: 0.758057
[600]	valid_0's auc: 0.758399
[700]	valid_0's auc: 0.758594
[800]	valid_0's auc: 0.758941
[900]	valid_0's auc: 0.758886
Early stopping, best iteration is:
[855]	valid_0's auc: 0.758982
Important features:
[('EXT_SOURCE_2', 1579), ('EXT_SOURCE_3', 1528), ('DAYS_BIRTH', 1385), ('EXT_SOURCE_1', 1245), ('AMT_ANNUITY', 1110), ('AMT_CREDIT', 1107), ('DAYS_ID_PUBLISH', 987), ('DAYS_EMPLOYED', 969), ('DAYS_LAST_PHONE_CHANGE', 942), ('DAYS_REGISTRATION', 826), ('AMT_GOODS_PRICE', 823), ('AMT_INCOME_TOTAL', 651), ('REGION_POPULATION_RELATIVE', 592), ('bureau_credit_active_count', 567), ('bureau_credit_closed_count', 389), ('OWN_CAR_AGE', 376), ('HOUR_APPR_PROCESS_START', 357), ('CODE_GENDER', 290), ('AMT_REQ_CREDIT_BUREAU_YEAR', 270), ('bureau_currency_count', 253), ('TOTALAREA_MODE', 2

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:55: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike


,SK_ID_CURR,TARGET
0,100001,0.030024
1,100005,0.105644
2,100013,0.027662
3,100028,0.027796
4,100038,0.158694


In [ ]:
prd_1 = pd.DataFrame(probs)

submit = pd.concat([test['SK_ID_CURR'],prd_1],axis=1)

print (submit.columns.tolist)

submit = submit.drop(submit.columns[1], axis=1)
#probs.head()
submit.to_csv('home-default-risk.csv',index=False,quoting=csv.QUOTE_NONNUMERIC)
submit.head()

In [ ]:
ax1 = sns.distplot(train["AMT_CREDIT"][train.TARGET==1], color='y')


In [ ]:

sns.kdeplot(train.loc[train['TARGET'] == 0, 'DAYS_BIRTH'], label = 'Repaid Loan')
sns.kdeplot(train.loc[train['TARGET'] == 1, 'DAYS_BIRTH'], label = 'Not Repaid Loan')
plt.xlabel('Age (years)')
plt.ylabel('Density')
plt.title('Distribution of Ages');